In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("train_loan_pred.csv")

In [11]:
data.Gender.value_counts()

Male      489
Female    112
Name: Gender, dtype: int64

In [12]:
data.shape

(614, 13)

In [14]:
data.isna().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [15]:
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [59]:
prep_data = data
data.Self_Employed.value_counts()

Male      489
Female    112
NA         13
Name: Self_Employed, dtype: int64

In [60]:
prep_data['Gender'] = data.Gender.fillna('NA')
prep_data['Married'] = data.Married.fillna('NA')
prep_data['Dependents'] = data.Dependents.fillna(0)
prep_data['Self_Employed'] = data.Self_Employed.fillna('NA')
prep_data['LoanAmount'] = data.LoanAmount.fillna(data.LoanAmount.dropna().mean())
prep_data['Loan_Amount_Term'] = data.Loan_Amount_Term.fillna(360.0)
prep_data['Credit_History'] = data.Credit_History.fillna(-1)

target_map = {"Y":1,"N":0}
prep_data.Loan_Status = prep_data.Loan_Status.map(target_map)

In [61]:
x_data = prep_data.drop(['Loan_Status'],axis = 1)
y_data = prep_data['Loan_Status']

In [62]:
x_data = x_data.drop(['Loan_ID'],axis = 1)

In [63]:
xp_data = pd.get_dummies(x_data)
xp_data.head()



,ApplicantIncome,CoapplicantIncome,LoanAmount,Gender_Female,Gender_Male,Gender_NA,Married_Female,Married_Male,Married_NA,Dependents_Female,...,Self_Employed_NA,Loan_Amount_Term_Female,Loan_Amount_Term_Male,Loan_Amount_Term_NA,Credit_History_Female,Credit_History_Male,Credit_History_NA,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
0,5849,0.0,146.412162,0,1,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,1
1,4583,1508.0,128.000000,0,1,0,0,1,0,0,...,0,0,1,0,0,1,0,1,0,0
2,3000,0.0,66.000000,0,1,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,1
3,2583,2358.0,120.000000,0,1,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,1
4,6000,0.0,141.000000,0,1,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,1


In [64]:
from sklearn.model_selection import train_test_split
x_train, x_test , y_train, y_test = train_test_split(xp_data,y_data)

In [65]:
y_data.value_counts()

1    422
0    192
Name: Loan_Status, dtype: int64

In [66]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 42)
o_x_train, o_y_train = smote.fit_resample(x_train,y_train)

In [67]:
o_y_train.value_counts()

1    306
0    306
Name: Loan_Status, dtype: int64

In [69]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators = 100, n_jobs = -1, random_state = 42)
rf_model.fit(o_x_train,o_y_train)


RandomForestClassifier(n_jobs=-1, random_state=42)

In [70]:
rf_prediction = rf_model.predict(x_test)


array([0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1],
      dtype=int64)

In [74]:
from sklearn.metrics import confusion_matrix, classification_report

labels = ["Negative","Positive"]
print(classification_report(y_test,rf_prediction))
pd.DataFrame(confusion_matrix(y_test,rf_prediction), index = labels, columns = labels)

              precision    recall  f1-score   support

           0       0.38      0.47      0.42        38
           1       0.81      0.75      0.78       116

    accuracy                           0.68       154
   macro avg       0.60      0.61      0.60       154
weighted avg       0.71      0.68      0.69       154



,Negative,Positive
Negative,18,20
Positive,29,87


In [76]:
from sklearn.feature_selection import SelectFromModel

sl_model = SelectFromModel(rf_model,prefit= True)
n_x_train  = sl_model.transform(o_x_train)
n_x_test  = sl_model.transform(x_test)

In [80]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators = 300, n_jobs = -1, random_state = 42)
rf_model.fit(n_x_train,o_y_train)
rf_prediction = rf_model.predict(n_x_test)


from sklearn.metrics import confusion_matrix, classification_report

labels = ["Negative","Positive"]
print(classification_report(y_test,rf_prediction))
pd.DataFrame(confusion_matrix(y_test,rf_prediction), index = labels, columns = labels)


              precision    recall  f1-score   support

           0       0.33      0.42      0.37        38
           1       0.79      0.72      0.75       116

    accuracy                           0.64       154
   macro avg       0.56      0.57      0.56       154
weighted avg       0.68      0.64      0.66       154



,Negative,Positive
Negative,16,22
Positive,33,83


In [ ]:
parameters = {
    'n_estimators' = [100,200,300,400,500],
    'max_depth' = [8,9,10,11,12],
    'min_samples_split' = [2,5,10]
}

from sklearn.model_selection import GridSearchCV

gs_model = GridSearchCV(RandomForestClassifier(), parameters, cv = 2, verbose = 1, n_jobs = -1)
gs_model.fit(n_x_train,o_y_train)

In [ ]:
print(gs_model.best_params_)
best_model = gs_model.best_estimator_
rf_prediction = best_model.predict(n_x_test)


from sklearn.metrics import confusion_matrix, classification_report

labels = ["Negative","Positive"]
print(classification_report(y_test,rf_prediction))
pd.DataFrame(confusion_matrix(y_test,rf_prediction), index = labels, columns = labels)

